In [1]:
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf

In [2]:
data = pd.read_csv("UCF-crime-frames-descriptions-partitions.csv")

In [3]:
data.head(2)

,Unnamed: 0,frame,directory,class,Description,partition
0,0,Abuse001_x264.mp4_frame_000000.jpg,Abuse_frames,Normal,a man is standing in the middle of a room,train
1,1,Abuse001_x264.mp4_frame_000010.jpg,Abuse_frames,Normal,a man is standing in the middle of a room,train


In [4]:
train_normal = data.loc[(data["partition"] == "train") & (data["class"] == "Normal")]
train_abnormal = data.loc[(data["partition"] == "train") & (data["class"] != "Normal")]

In [5]:
val_normal = data.loc[(data["partition"] == "validation") & (data["class"] == "Normal")]
val_abnormal = data.loc[(data["partition"] == "validation") & (data["class"] != "Normal")]

In [6]:
train_normal.shape, train_abnormal.shape

((84684, 6), (39461, 6))

In [7]:
val_normal.shape, val_abnormal.shape

((12895, 6), (6435, 6))

In [53]:
train = pd.concat([train_normal[0:2593], train_abnormal])
val = pd.concat([val_normal[0:646], val_abnormal])

In [54]:
def get_data(df):
    X = np.zeros((len(df), 1408))
    path = "d:/data/UCF-crime/Anomaly-Videos-frames-features"
    c = 0
    for _, f in df.iterrows():
        directory = f["directory"][:-7]
        X[c] = np.load(f"{path}/{directory}/{f['frame']}.npy")
        c += 1    
    y = list(df["class"])
    y = [labels[lbl] for lbl in y]    
    return X, np.array(y)

In [55]:
labels = {"Normal": 0,
          "Abuse": 1,
          "Arrest": 2,
          "Arson": 3,
          "Burglary": 4,
          "Explosion": 5,
          "Fighting": 6,
          "RoadAccidents": 7,
          "Shooting": 8,
          "Vandalism": 9
         }

In [92]:
V, y_val = get_data(val)

In [93]:
V.shape, y_val.shape

((1292, 1408), (1292,))

In [94]:
X, y = get_data(train)

In [95]:
X.shape, y.shape

((5186, 1408), (5186,))

In [60]:
m, M, mean = tf.reduce_min(X), tf.reduce_max(X), tf.reduce_mean(X)
X = (X-mean)/(M-m)
V = (V-mean)/(M-m)

In [96]:
tf.reduce_min(X), tf.reduce_max(X), tf.reduce_mean(X)

(<tf.Tensor: shape=(), dtype=float64, numpy=-19.09375>,
 <tf.Tensor: shape=(), dtype=float64, numpy=12.328125>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-0.0017183497589734168>)

In [97]:
tf.reduce_min(V), tf.reduce_max(V), tf.reduce_mean(V)

(<tf.Tensor: shape=(), dtype=float64, numpy=-18.03125>,
 <tf.Tensor: shape=(), dtype=float64, numpy=11.6328125>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-0.00029143080352670564>)

In [16]:
from sklearn.utils import class_weight
_, _, counts = tf.unique_with_counts(y)
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = dict(enumerate(weights))

In [110]:
model = Sequential()
model.add(Dense(10, activation="softmax", input_shape=(1408,)))
model.add(Dropout(0.5))
#model.add(Dense(64, activation="relu"))
#model.add(Dropout(0.5))
#model.add(Dense(16, activation="relu"))
#model.add(Dropout(0.5))
#model.add(Dense(10, activation="softmax"))

In [111]:
optimizer = Adam(learning_rate=.00001)
model.compile(optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [112]:
history = model.fit(X, y, 
                    validation_data=(V, y_val), 
                    shuffle=True,
                    #class_weight=class_weight_dict, 
                    epochs=10, batch_size=64)

Epoch 1/10
82/82 [==============================] - 1s 6ms/step - loss: 3.3084 - accuracy: 0.4321 - val_loss: 6.9746 - val_accuracy: 0.5008
Epoch 2/10
82/82 [==============================] - 0s 3ms/step - loss: 2.1457 - accuracy: 0.4790 - val_loss: 6.8554 - val_accuracy: 0.5008
Epoch 3/10
82/82 [==============================] - 0s 2ms/step - loss: 1.9710 - accuracy: 0.4703 - val_loss: 7.2919 - val_accuracy: 0.5008
Epoch 4/10
82/82 [==============================] - 0s 2ms/step - loss: 1.8748 - accuracy: 0.4786 - val_loss: 7.4606 - val_accuracy: 0.5008
Epoch 5/10
82/82 [==============================] - 0s 2ms/step - loss: 1.7938 - accuracy: 0.4855 - val_loss: 7.8270 - val_accuracy: 0.5008
Epoch 6/10
82/82 [==============================] - 0s 2ms/step - loss: 1.8058 - accuracy: 0.4769 - val_loss: 7.6816 - val_accuracy: 0.5008
Epoch 7/10
82/82 [==============================] - 0s 2ms/step - loss: 1.7583 - accuracy: 0.4830 - val_loss: 7.8218 - val_accuracy: 0.5008
Epoch 8/10
82/82 [==

In [113]:
val_pred = model.predict(V)
print(classification_report(y_val, tf.argmax(val_pred, axis=1)))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00       646
           1       0.50      1.00      0.67       646

    accuracy                           0.50      1292
   macro avg       0.75      0.50      0.34      1292
weighted avg       0.75      0.50      0.34      1292



In [114]:
tf.argmax(val_pred[0:1000], axis=1)

<tf.Tensor: shape=(1000,), dtype=int64, numpy=
array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
    

In [115]:
y_pred = model.predict(X)
print(classification_report(y, tf.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      2593
           1       0.94      0.91      0.93      2593

    accuracy                           0.93      5186
   macro avg       0.93      0.93      0.93      5186
weighted avg       0.93      0.93      0.93      5186



In [109]:
tf.argmax(y_pred[2000:3000], axis=1)

<tf.Tensor: shape=(1000,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
    